In [90]:
import requests
import json
import pandas as  pd
import time
# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

In [91]:
url = 'http://127.0.0.1:5000/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [92]:
response.json()

{'signingKey_hex': '8d7aacdba00f275e39aa448367eeb1889a0855a0ff0e158f68afec97161b17ae',
 'traderId': '1',
 'verifyKey_hex': 'acefc9cc3b1e5313574e8bd12cbf33fc1327904b24e1d1212d3a242efbac0d2d'}

In [93]:
url = 'http://127.0.0.1:5000/createMarket'

mkId = 11
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1.099}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [94]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 1.099,
 'marketMin': 0,
 'marketRootId': 11,
 'traderId': '1'}

In [95]:
tId

'1'

In [96]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 1.099,
 'marketMin': 0,
 'marketRootId': 11,
 'traderId': '1'}

In [97]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1.099,0,11,1


In [98]:
# Create a trade
url = 'http://127.0.0.1:5000/createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.512, "quantity":1}

response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [99]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 11,
 'price': 0.512,
 'quantity': 1,
 'traderId': '1'}

In [100]:
# Create a matched trade
url = 'http://127.0.0.1:5000/createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [101]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 11,
 'price': 0.512,
 'quantity': -1,
 'traderId': '1'}

In [103]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 1.1941938400268555 seconds.
matched trade: 1 in 1.2259860038757324 seconds.
matched trade: 2 in 1.2691669464111328 seconds.
matched trade: 3 in 1.2726032733917236 seconds.
matched trade: 4 in 1.2234830856323242 seconds.
matched trade: 5 in 1.2161049842834473 seconds.
matched trade: 6 in 1.232724905014038 seconds.
matched trade: 7 in 1.3047480583190918 seconds.
matched trade: 8 in 1.3405778408050537 seconds.
matched trade: 9 in 1.2951180934906006 seconds.


In [104]:
url = 'http://127.0.0.1:5000/createMarket'

for i in range(3):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 0.17719388008117676 seconds.
market created: 2 in 0.2695159912109375 seconds.
market created: 3 in 0.5149691104888916 seconds.


In [105]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 3,
 'traderId': 1}

In [106]:
url = 'http://127.0.0.1:5000/viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [107]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,11,1,0,1.099
1,1,1,0,0.700
2,2,1,0,0.700
3,3,1,0,0.700


In [108]:
url = 'http://127.0.0.1:5000/viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [110]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,tradeRootId,traderId
0,11,1,0.512,1,1,1
1,11,1,0.512,-1,2,1
10,11,1,0.512,1,3,1
100,11,1,0.512,1,33,1
101,11,1,0.512,-1,33,1
102,11,1,0.512,-1,35,1
103,11,1,0.512,1,36,1
104,11,1,0.512,-1,36,1
105,11,1,0.512,1,36,1
106,11,1,0.512,1,35,1
